In [2]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv()

src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [15]:
import pandas as pd
from sqlalchemy import create_engine

# замените пользователя, пароль, хост, порт и имя базы данных на реальные учётные данные общей базы данных

src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

# определяем функцию получения данных из первичного источника
# она получает на вход объект соединения к первичному источнику
# и возвращает данные из всех таблиц, собранные в одном pandas.DataFrame
def extract(cnx) -> pd.DataFrame:
  # сначала напишите SQL-запрос, который объединяет все таблицы в одну
  sql_query = f"""
                select c.customer_id,
                c.begin_date,
                c.end_date,
                c."type",
                c.paperless_billing,
                c.payment_method,
                c.monthly_charges,
                c.total_charges,
                i.internet_service,
                i.online_security,
                i.online_backup,
                i.device_protection,
                i.tech_support,
                i.streaming_tv,
                i.streaming_movies,
                p.gender,
                p.senior_citizen,
                p.partner,
                p.dependents,
                p2.multiple_lines
                from contracts c 
                left join internet i on c.customer_id  = i.customer_id 
                left join personal p on c.customer_id = p.customer_id 
                left join phone p2 on c.customer_id = p2.customer_id    
        """
  data = pd.read_sql(sql_query, cnx) #исполним написанный запрос
  return data

def transform(data: pd.DataFrame) -> pd.DataFrame:

  # Мое решение
  # data["target"] = 1
  # data["target"][data["end_date"] == "No"] = 0
  # data["end_date"][data["end_date"] == "No"] = None

  # Решение курса
  # Для создания колонки target нужно сначала получить колонку, состоящую из True/False, где True означает, что пользователь перестал пользоваться услугами компании. А затем преобразовать True/False в int.
  data['target'] = (data['end_date'] != 'No').astype(int)
  data['end_date'].replace({'No': None}, inplace=True)

  return data

def load(data: pd.DataFrame, db_conn: sqlalchemy.engine.base.Engine, table_name: str = 'users_churn'):
  data.to_sql(table_name, db_conn)

In [ ]:
df = extract(src_conn)
transformed_data = transform(df)
load(transformed_data, dst_conn)